In [ ]:
import numpy as np
import pickle
from paths import *
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pickle
from paths import *
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM
from keras.layers import  GRU
from create_embedding import *
from helper import *
from create_emebedding import *
from paths import *
from tweet_vectors import *
from keras.layers import Bidirectional

In [ ]:
def build_dl_model():
    print('preprocessing')
    X_pos, X_neg,  X_test = load_cleaned_data_and_test(DATASETS_PATH + 'train_pos_full.txt', \
                   DATASETS_PATH + 'train_neg_full.txt', DATASETS_PATH + 'test_data.txt' ,HASHTAG = True, EMPHASIZE = True, PUNC=True, NUMBER =True, SMALL_WORDS = True , \
                        SLANG =True, APOSTROPHE = True, EMOJI = False, REPITITION = True , SPELLING = True, \
                        STOPWORDS = False, LEMMATIZE = False, STEMMING = False)
    X = X_pos + X_neg
    #save full cleaned datasets
    pickle.dump(X, open(CLEANED_DATA_PATH + 'full_X_Cleaned.pkl', 'wb'))
    pickle.dump(X_test, open(CLEANED_DATA_PATH + 'full_X_test_Cleaned.pkl', 'wb'))
    print('using values 200 and 30 respectively for dimension and max_length')
    dimension = 200
    MAX_LENGTH = 30
    sequence_X , sequence_test , embedding_matrix , y , vocab_size = prepare_DL_input(X , X_test , dimension , MAX_LENGTH )
    #put best model here 
    print('building model')
    model = Sequential()
    model.add(Embedding(vocab_size, dimension, input_length=MAX_LENGTH, weights=[embedding_matrix]))
    model.add(Bidirectional(LSTM(100, dropout=0.2)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('training model')
    model.fit(sequence_X, y, epochs=2, batch_size=128, verbose=1, shuffle=True)
    print('prediction')
    pred = model.predict_proba(sequence_test)  
    new_pred = [1 if x <= 0.5 else -1 for x in pred]
    ids = np.arange(1,10001)
    name = 'BestModel.csv'
    create_csv_submission(ids,new_pred,name)